# Tarea.  Author Profiling


Cipriano C. Hdz. (Matemáticas Aplicadas)

-------------------------------------------------------

El siguiente código está basado en los correspondientes de Luis Pardo:

Simple RNN y RNN+ atención (Visualización)

https://colab.research.google.com/drive/1z7OqJTUcwiVeAxDmf_Hefz4rmLdwZKGc?usp=sharing#scrollTo=ehRrS58B4jLq

Atención Jeráquica

https://colab.research.google.com/drive/1mamwPAQw4h8RkwLGOnuH4S04zLemms87?authuser=2

------------------------------------------------------------

Descargamos los datos

In [ ]:
#!gdown --i  1UcbTsXgVgEulZKHdpzaAjIC0I6HQA0vV

In [ ]:
#!unzip /content/author_profiling_pan.zip

Descargamos word2vec

In [ ]:
# from google_drive_downloader import GoogleDriveDownloader as gdd

In [ ]:
'''gdd.download_file_from_google_drive(file_id   = '1i5f_35i_sSIQ0ny6MAEIMzTDmnxiSsvd',
                                    dest_path = '/content/word2vec_col.txt',
                                    unzip     = False)'''

"gdd.download_file_from_google_drive(file_id   = '1i5f_35i_sSIQ0ny6MAEIMzTDmnxiSsvd',\n                                    dest_path = '/content/word2vec_col.txt',\n                                    unzip     = False)"

Librerías 

In [ ]:
import pandas as pd
import pickle
import numpy as np
import nltk
nltk.download('punkt')
from tqdm.auto import tqdm
import copy

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence, pack_padded_sequence
import torch.nn.functional as F

from sklearn.metrics import f1_score


import xml.etree.ElementTree as ET
import glob
import os
import re

from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to /home/esteban/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/esteban/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1) Haga el preprocesamiento que usted considere para limpiar y extraer la información de los documentos

Tratamiento de los datos.

Primero debemos de transformar los archivos, donde cada archivo representa un autor, y dentro de este se encuentre sus respectivos tweets.

Como cada archivo es un .xml, parseamos cada archivo.

Ejemplo:

In [ ]:
tree = ET.parse('./author_profiling_pan/es_train/100f11229a97c38f95e0a563039af29e.xml')
root = tree.getroot()

Observamos los tweets de un solo usuario

In [ ]:
tweet=''
for elem in root:
    for subelem in elem:
        tweet = tweet+' '+ subelem.text
        print(subelem.text)

¿Os habéis perdido el directo de hoy? ¡Puedes verlo ahora! https://t.co/8BUWSZJ8zZ
¡Muchas gracias por ver mi vídeo de hoy en directo en YouTube! Empecemos bien la semana ;) Let's… https://t.co/NqR95v82ON
En 5 minutos comienzo a emitir en directo https://t.co/8BUWSZrxIr ¡Acompáñame! RT!!
DIRECTO: ¿QUÉ ME ESTÁ PASANDO? https://t.co/IL0rxR1lum vía @YouTube Hoy a las 8:00 pm (hora Madrid) os espero &lt;3
Hace frío en Madrid... Y mientras mindeliciousme está en el Caribe :( https://t.co/G0hJI5TRqj
@RodrigoSeptien Un placer haber coincidido contigo, artista!! Espero que en algún momento podamos colaborar juntos! Un abrazo!
¡Sorpresa! Esta semana, ¡doble vídeo! (Link en bio)
Hoy he ido a la presentación de #TubeconES… https://t.co/GtmtCV39ij
¡Sorpresa! esta semana publico doble vídeo con motivo de la presentación de TUBECON #tubecones donde... https://t.co/zHrF8Aijwc
No estoy pasando buenos momentos en lo personal, pero siempre intento llevar la sonrisa puesta.… https://t.co/b6DLSifQHp
@Emma

In [ ]:
tweet

" ¿Os habéis perdido el directo de hoy? ¡Puedes verlo ahora! https://t.co/8BUWSZJ8zZ ¡Muchas gracias por ver mi vídeo de hoy en directo en YouTube! Empecemos bien la semana ;) Let's… https://t.co/NqR95v82ON En 5 minutos comienzo a emitir en directo https://t.co/8BUWSZrxIr ¡Acompáñame! RT!! DIRECTO: ¿QUÉ ME ESTÁ PASANDO? https://t.co/IL0rxR1lum vía @YouTube Hoy a las 8:00 pm (hora Madrid) os espero &lt;3 Hace frío en Madrid... Y mientras mindeliciousme está en el Caribe :( https://t.co/G0hJI5TRqj @RodrigoSeptien Un placer haber coincidido contigo, artista!! Espero que en algún momento podamos colaborar juntos! Un abrazo! ¡Sorpresa! Esta semana, ¡doble vídeo! (Link en bio)\nHoy he ido a la presentación de #TubeconES… https://t.co/GtmtCV39ij ¡Sorpresa! esta semana publico doble vídeo con motivo de la presentación de TUBECON #tubecones donde... https://t.co/zHrF8Aijwc No estoy pasando buenos momentos en lo personal, pero siempre intento llevar la sonrisa puesta.… https://t.co/b6DLSifQHp @E

In [ ]:
dir_test = './author_profiling_pan/es_test'
dir_train  = './author_profiling_pan/es_train'

In [ ]:
input_list

['./author_profiling_pan/es_train/100f11229a97c38f95e0a563039af29e.xml',
 './author_profiling_pan/es_train/10111cf67718994daaff92ff59bc679f.xml',
 './author_profiling_pan/es_train/1011b55a13502bcb7562d610a05ef3bd.xml',
 './author_profiling_pan/es_train/1026000a7c186f3a4aba1053f9956b42.xml',
 './author_profiling_pan/es_train/1032e9d0da6d21ceab370759d38930a9.xml',
 './author_profiling_pan/es_train/1062f0bc40b27f98bd6775b4cb5a0ab3.xml',
 './author_profiling_pan/es_train/108777384be3c97767b02ea4e24760f5.xml',
 './author_profiling_pan/es_train/10880d19d7346373a42b18e505bbc4d5.xml',
 './author_profiling_pan/es_train/109017200e3161969f8be1b6f54db888.xml',
 './author_profiling_pan/es_train/10aef88d048dafbc75639241b9e35df8.xml',
 './author_profiling_pan/es_train/10bb37101c79cab0cc07b69d4fb67707.xml',
 './author_profiling_pan/es_train/10cb6c144d8d763ab4ec9685dfced242.xml',
 './author_profiling_pan/es_train/10d1d084bb0c0550df10c8cd49cbe86d.xml',
 './author_profiling_pan/es_train/10fd008c09eb119fe

Cargamos las etiquetas correspondientes al género.

In [ ]:
input_list[-1]

'./author_profiling_pan/es_train/truth.txt'

In [ ]:
labels = pd.read_csv('./author_profiling_pan/es_train/truth.txt', sep=':::', engine='python',  header = None, names = ['Id', 'Gender'], usecols=[0,1])

In [ ]:
labels

,Id,Gender
0,74bcc9b0882c8440716ff370494aea09,female
1,4639c055f34ca1f944d0137a5aeb7914,female
2,92ffa98bade702b86417b118e8aca319,female
3,4560c6567afcccef265f048ed117d04d,female
4,393866dfaa80d414c9896cf8723932b7,female
...,...,...
4195,b93801b0abc44cb3d45e5f9cd838bcbd,male
4196,737923304de3e95ff95f8e0be537a2d4,male
4197,acb8ad6e2b8b5408526469f1fa37219a,male
4198,458afd67845a7747ba98b540404f87d9,male


La siguiente instrución cambia Mujer por 1 y Hombre a 0

Ahora transformamos los hombres y mujeres en 1,0, respectivamente

In [ ]:
labels['Gender'] = labels['Gender'].map({'female': 1, 'male': 0})

In [ ]:
labels

,Id,Gender
0,74bcc9b0882c8440716ff370494aea09,1
1,4639c055f34ca1f944d0137a5aeb7914,1
2,92ffa98bade702b86417b118e8aca319,1
3,4560c6567afcccef265f048ed117d04d,1
4,393866dfaa80d414c9896cf8723932b7,1
...,...,...
4195,b93801b0abc44cb3d45e5f9cd838bcbd,0
4196,737923304de3e95ff95f8e0be537a2d4,0
4197,acb8ad6e2b8b5408526469f1fa37219a,0
4198,458afd67845a7747ba98b540404f87d9,0


Ahora retiramos todas los hipervínculos que se encuentran en cada tweet, así como las menciones @usuario.

Ejemplo:

In [ ]:
text = 'Ayer @rolandoruizdura en #Ávila: https://t.co/hf7RIxirDI "Hay q arrodillarse ant los alumnos y servirlos, como hizo… https://t.co/mD3LsknSa1'
print(text)

Ayer @rolandoruizdura en #Ávila: https://t.co/hf7RIxirDI "Hay q arrodillarse ant los alumnos y servirlos, como hizo… https://t.co/mD3LsknSa1


In [ ]:
text_dict = {}
for i, document in enumerate(input_list[:-1]):
    name = input_list[:-1][i].split('/')[-1][:-4]
    tree = ET.parse(document)
    root = tree.getroot()
    tweet=''
    for elem in root:
        for subelem in elem:
            tweet = tweet+' '+ subelem.text
    text_dict[name] = tweet

Las expresiones regulares usadas fueron obtenidas de los siguientes sitios:

In [ ]:
Las expresiones regulares usadas fueron obtenidas de los siguientes sitios:

Ayer @user en #Ávila:  "Hay q arrodillarse ant los alumnos y servirlos, como hizo… 


https://stackoverflow.com/questions/2304632/regex-for-twitter-username

https://stackoverflow.com/questions/6038061/regular-expression-to-find-urls-within-a-string

In [ ]:
text = re.sub(r'(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])','', text)
text = re.sub(r'@([A-Za-z0-9_]+)','@user', text)
print(text)

 no entiendo, doy los mejores consejos (?), pero nunca se que hacer con mi vida 🤔  tengo una extraña sensación en la que extraño mucho a nella 🤔 @user @user jajajajaja es cierto 🙈 @user me extrañaste, lo se 😌 @user yo solo me mecho contigo en las juergas jajajaja no se si soy yo o qué, pero casi nada me enoja o jode 🤔 he tenido el mejor sábado de la vida qué rico universitario estoy viendo 😍 tengo un culo que leer pero sigo chorreada en mi cama. típico de mi @user aventarse es más fácil ver makis y empezar a salivar en exceso. típico de mi gracias temblor, cómo se supone ahora que vuelva a coger sueño? tengos unas ganas locas de leche asada 😩 el amiguito de piero se va a mudar y se han puesto a llorar porque se van a extrañar jajajaja 😂💕 @user como te gusta la cochinada no? jajaja te confesé que quiero meterte en mi cama viéndote sonreír de placer ahora hace calor y la música suena bastante bien si te mueres mañana, no te quedes con ganas de nada @user yo también 🙊 felicidad x1000000 s

La siguiente función procesa los datos, esto no es más que una versión editada de la correspondiente funcioń de Luis Pardo.

In [ ]:
class tweet_dataset(Dataset):
    def __init__(self, directory):
        super(Dataset, self).__init__()
        self.load_data(directory)
        self.vocab, self.emb_mat = self.load_vocab_embeddings()
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        '''Método principal para cargar una observación del dataset.
           label: categoría a la que pertenece la observación.
           word_ids: lista de índices de las palbras en el vocabulario.
        '''
        label = self.data.iloc[index]['Gender']
        words, word_ids = self.preprocessed_text(index)
        return word_ids, label, words
        
    def preprocessed_text(self, index):
        '''Preprocess text and '''

        stop_words=stopwords.words('spanish')
        stop_words.extend(['.',',','?','¿','!','¡',':',';','$','&','%','/'])

        text = self.data.iloc[index]['Text']
        words = nltk.word_tokenize(text)
        words = [ word for word in words if word not in stop_words ]
        word_ids = [self.vocab[word] if word in self.vocab.keys() else self.emb_mat.shape[0]-1 for word in words]
        return words, word_ids

    def load_data(self, directory):
        '''Método para cargar datos.
           El texto está en la columna "text" y las categorías en la columna "target".
        '''
        #Enlistamos todas las rutas de los archivos
        input_list = sorted([os.path.join(directory, name) for name in os.listdir(directory)])
        #Obtenemos las etiquetas de Mujer/Hombre
        labels=input_list[-1]
        df = pd.read_csv(labels, sep = ':::', header = None, engine='python', names = ['Id', 'Gender'], usecols=[0,1])
        #Hacemos la transformación Mujer: 1 y Hombre: 0
        df['Gender'] = df['Gender'].map({'female': 1, 'male': 0})
        #Obtenemos los tweets dadas sus direcciones
        lista_tweets=input_list[:-1]
        text_dictionary = self.get_tweets(lista_tweets)    # Regresamos el diccionario (nombre archivo, cadena de tweets)
        df['Text']      = df['Id'].map(text_dictionary)   # Hacemos un join con el dataframe que tiene las etiquetas
        self.data       = df                              

    def load_vocab_embeddings(self):
        '''Embeddings preentrenados en twitter.
           emb_mat: Matriz de embeddings. Un vector de tamaño 200 para cada palabra del vocabulario.
           vocab: Diccionario, asigna a cada palabra su renglón correspondiente en la matriz de embeddings.
        '''
        embeddings_list = []
        self.vocab_dict = {}
        vocab = {}
        with open('./word2vec_col.txt', 'r') as f:
            for i, line in enumerate(f):
                if i!=0:
                    values = line.split()
                    self.vocab_dict[i+1] = values[0]
                    vocab[values[0]] = i+1
                    vector = np.asarray(values[1:], "float32")
                    embeddings_list.append(vector)
        embeddings_list.insert(0,np.mean(np.vstack(embeddings_list), axis=0))
        embeddings_list.insert(0,np.zeros(100))
        self.vocab_dict[0] = '[PAD]'
        self.vocab_dict[1] = '[UNK]'
        vocab['[PAD]'] = 0
        vocab['[UNK]'] = 1
        emb_mat = np.vstack(embeddings_list)

        return vocab, emb_mat

    def get_weights(self):
        '''Devuelve pesos inversos para cada categoría. Mayor peso para la categoría con menos observaciones.'''

        cat_0 = len(self.data[self.data['Gender']==0])
        cat_1 = len(self.data[self.data['Gender']==1])
        maxi = max(cat_0, cat_1)
        return torch.tensor([maxi/cat_0, maxi/cat_1])

    def get_tweets(self,input_list):
        # Creamos diccionario para tweets
        text_dict = {}
        for i, document in enumerate(input_list):
            #Obtenemos el nombre el archivo, respectivo a la posición i
            name = input_list[i].split('/')[-1][:-4]
            #Parseamos el documento xml, y lo tratamos
            tree = ET.parse(document)
            root = tree.getroot()
            #Guardaremos el tweet en una sola cadena
            tweet=''
            for elem in root:
                for subelem in elem:
                    tweet = tweet+' '+ subelem.text
                    #procesamos el tweet
                    #Retiramos todas las URLS que contenga
                    tweet = re.sub(r'(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])','', tweet)
                    #Enmascaramos todos los usuarios etiquetados mediante el pad @user
                    tweet = re.sub(r'@([A-Za-z0-9_]+)','@user', tweet)
            #Retornamos un diccionario de la forma (nombre documento, tweets del documento)
            text_dict[name] = tweet
        return text_dict

    def collate_fn(self, batch):
        '''Función que ejecuta el dataloader para formar batches de datos.'''

        zipped_batch = list(zip(*batch))
        word_ids = [torch.tensor(t) for t in zipped_batch[0]]
        word_ids = torch.cat(word_ids, dim=0)
        lengths = torch.tensor([len(t) for t in zipped_batch[0]])
        labels = torch.tensor(zipped_batch[1])
        words = zipped_batch[2]
        return word_ids, lengths, labels, words

Ahora si, pasamos los archivos por la función anterior:

In [ ]:
dir_test = './author_profiling_pan/es_test'
dir_train  = './author_profiling_pan/es_train'

In [ ]:
train_dataset = tweet_dataset(dir_train)
test_dataset  = tweet_dataset(dir_test)

Observamos la base de datos obtenida:  notemos que el nombre el aechivo se convierte en el id para identificar a los autores, y hemos hecho el join con los tweets de cada uno.


In [ ]:
train_dataset.data

,Id,Gender,Text
0,74bcc9b0882c8440716ff370494aea09,1,Tiene que valer la pena que esté despierta a ...
1,4639c055f34ca1f944d0137a5aeb7914,1,Eres lo mas importante para mi. 💜D💜 Lo que no...
2,92ffa98bade702b86417b118e8aca319,1,@user no sólo la infraestructura también la s...
3,4560c6567afcccef265f048ed117d04d,1,@user @user en el cielo. Tu novio el que come...
4,393866dfaa80d414c9896cf8723932b7,1,No puedo con Canada. Esta muy gorda. Si quier...
...,...,...,...
4195,b93801b0abc44cb3d45e5f9cd838bcbd,0,Creo que nunca va existir nada mejor que abra...
4196,737923304de3e95ff95f8e0be537a2d4,0,Te necesito aquí conmigo. @user Estamos igual...
4197,acb8ad6e2b8b5408526469f1fa37219a,0,Pues ya no sé si voy a ir a Monterrey o no......
4198,458afd67845a7747ba98b540404f87d9,0,"""Fulanito"" ha aceptado tu solicitud de amista..."


Veamos los tokens:

In [ ]:
word_ids, label, words = train_dataset.__getitem__(1)

In [ ]:
print(words)

['Eres', 'mas', 'importante', '💜D💜', 'Lo', 'mata', 'hace', 'mas', 'fuerte', '😊💪💪', 'Querer', 'dejar', 'poder', 'desaparecer', '😑😑😑', 'Él', 'puesto', 'tres', 'puntos', 'suspensivos', 'historia', '...', 'Ella', 'borró', 'dos', '``', 'Ámame', 'menos', 'merezca', 'mas', 'necesito', "''", 'Nada', 'mas', 'bonito', 'comprar', 'ropa', 'contigo', 'soportes', 'hecho', 'gusta', 'Jaja', 'Irse', 'mas', 'lejos', 'posible', '😑😞😞😞😔🚀', 'Te', 'prometo', 'corazón', '✋💜', 'Trabajo', 'investigación', '😵😵😵', 'Último', 'parcial', '😁😁😁', '@', 'user', 'jajaja', 'frío', 'mata', 'frío', 'jaaja', '@', 'user', 'jajaja', 'ashh', 'dijiste', 'veía', 'tan', 'mal', '😮😔😭', 'Te', 'extraño', '😔😔', 'inevitable', 'Trabajo', 'investigación', 'foro', 'ensayo', 'trabajo', 'final', '😵', 'creo', 'pasare', 'largo', '😩😪', 'Dañe', 'porqué', 'pensé', 'bien', 'cosas', '😞😞😢😢😢', 'Hasta', 'fin', 'dejo', 'orgullo', 'lado', '😊😆🙌👏', 'Tengo', 'mente', 'lugar', '😔😔😞', '🌌🌌', 'Y', 'si', 'bueno', 'termina', 'malo', 'Porque', 'amor', 'prometió',

Notamos que aún hay diferentes caracteres que podrían eliminarse.

# RNN SIN Atención

## 2) Diseñe un clasificador con alguna RNN que codifique cada perfil de usuario con todos sus tweets y clasifique

In [ ]:
batch_size = 10

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn = train_dataset.collate_fn, shuffle=True)
test_dataloader  = DataLoader(test_dataset, batch_size=batch_size, collate_fn = test_dataset.collate_fn, shuffle=False)

La siguiente función sigue siendo la misma propuesta por Luis Pardo:

In [ ]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size=100, hidden_size=128, num_layers=1,
                 bidirectional=False, emb_mat=None, dense_hidden_size=256):
        '''Constructor, aquí definimos las capas.
        input:
            input_size: Tamaño de los embeddings de las palabras.
            hidden_size: Tamaño de la capa oculta de la GRU.
            num_layers: Número de capas de la GRU.
            bidirectional: True si se quiere una GRu bidireccional.
            emb_mat: Matriz de embeddings del vocabulario.
            dense_hidden_size: Tamaño de la capa ocula del clasificador.
        '''
        super(SimpleRNN, self).__init__()
        # Matriz entrenable de embeddings, tamaño vocab_size x 100
        self.embeddings = nn.Embedding.from_pretrained(\
                            torch.FloatTensor(emb_mat), freeze=False)
        # Gated Recurrent Unit
        self.gru = nn.GRU(input_size=input_size, hidden_size=hidden_size, 
                          num_layers=num_layers, bidirectional=bidirectional)
        # Número de direcciones de la GRU
        directions = 2 if bidirectional else 1
        # Clasificador MLP
        self.classifier = nn.Sequential(\
                            nn.Linear(hidden_size*directions, dense_hidden_size),
                            nn.BatchNorm1d(dense_hidden_size),
                            nn.ReLU(),
                            nn.Linear(dense_hidden_size, 2))
    
    def forward(self, input_seq, lengths):
        '''Función feed-forward de la red.
        input:
            input_seq: Lista de ids para cada palabra.
            lengths: Número de palabras en cada una de las observaciones del batch.
        output:
            x: vectores para clasificar.
            return None for consistency with the next model
        '''
        # Calcula el embedding para cada palabra.
        x = self.embeddings(input_seq)
        # Forma las secuencias de palabras que entraran a la GRU.
        x = x.split(lengths.tolist())
        # Añade pading y empaqueta las secuencias (mayor velocidad de cómputo).
        x = pad_sequence(x)
        x = pack_padded_sequence(x, lengths, enforce_sorted=False)
        output, hn = self.gru(x)
        hn = torch.cat([h for h in hn], dim=-1)
        x = self.classifier(hn)
        return x, None

In [ ]:
def eval_model(model, dataloader, criterion, device):
    '''Función para evaluar el modelo.'''
    with torch.no_grad():
        model.eval()
        losses = []
        preds = torch.empty(0).long()
        targets = torch.empty(0).long()
        scores_list = []
        words_list = []
        pred_list = []
        for data in tqdm(dataloader):
            torch.cuda.empty_cache()
            seq, seq_len, labels, words = data
            seq, labels = seq.to(device), labels.to(device)
            output, scores = model(seq, seq_len)
            output = F.log_softmax(output, dim=1)
            loss = criterion(output, labels)
            losses.append(loss.item())
            predictions = F.log_softmax(output, dim=1).argmax(1)
            preds = torch.cat([preds, predictions.cpu()], dim=0)
            targets = torch.cat([targets, labels.cpu()], dim=0)
            if scores is not None:
                pred_list += predictions.tolist()
                scores = scores.cpu().squeeze(2).tolist()
                scores_list += scores
                words_list += words

        model.train()
        preds = preds.numpy()
        targets = targets.numpy()
        f1 = f1_score(targets, preds, average='binary')
        acc = accuracy_score(targets, preds)
        return np.mean(losses), f1, scores_list, words_list, pred_list,acc

In [ ]:
lr = 0.001
epochs = 10
weight_decay=0.0001
beta1=0
beta2=0.999
device = torch.device('cuda')

In [ ]:
model = SimpleRNN(emb_mat=train_dataset.emb_mat, bidirectional=False).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
weight = train_dataset.get_weights().to(device)
criterion = nn.NLLLoss(weight = weight)

In [ ]:
best_val_f1 = 0
for epoch in range(epochs):
    for data in tqdm(train_dataloader):
        # Limpia basura de la memoria GPU
        torch.cuda.empty_cache()
        # Reiniciamos el cálculo del gradiente
        optimizer.zero_grad()
        # Desempaca los datos que salen del dataloader
        seq, seq_len, labels, _ = data
        # Mueve los datos al mismo device en el que este el modelo
        seq, labels = seq.to(device), labels.to(device)
        output, _ = model(seq, seq_len)
        output = F.log_softmax(output, dim=1)
        loss = criterion(output, labels)
        # Calcula el gradiente de la pérdida
        loss.backward()
        # Realiza un paso de la optimización
        optimizer.step()
    
    #Evalúa los modelos en los conjuntos de entrenamiento y valuación
    train_loss, train_f1, _, _, _, train_accuracy = eval_model(model, train_dataloader, criterion, device)
    test_loss, test_f1, _, _, _, test_accuracy = eval_model(model, test_dataloader, criterion, device)
    print('epoch: %d'%(epoch))
    print('train_loss: %5f | train_f1: %5f | train_acc: %5f'%(train_loss, train_f1, train_accuracy)) 
    print('test_loss: %5f |  test_f1: %5f | test_acc: %5f'%(test_loss, test_f1, test_accuracy)) 
    if test_f1>best_val_f1:
        best_val_f1=test_f1
        best_state_dict=copy.deepcopy(model.state_dict())

  0%|          | 0/420 [00:00<?, ?it/s]

  0%|          | 0/420 [00:00<?, ?it/s]

  0%|          | 0/280 [00:00<?, ?it/s]

epoch: 0
train_loss: 0.632851 | train_f1: 0.694543 | train_acc: 0.629524
test_loss: 0.687050 |  test_f1: 0.647277 | test_acc: 0.558214


  0%|          | 0/420 [00:00<?, ?it/s]

...

epoch: 8
train_loss: 0.037085 | train_f1: 0.991445 | train_acc: 0.991429
test_loss: 1.415130 |  test_f1: 0.596888 | test_acc: 0.592857


  0%|          | 0/420 [00:00<?, ?it/s]

  0%|          | 0/420 [00:00<?, ?it/s]

  0%|          | 0/280 [00:00<?, ?it/s]

epoch: 9
train_loss: 0.037030 | train_f1: 0.987025 | train_acc: 0.987143
test_loss: 1.495397 |  test_f1: 0.581738 | test_acc: 0.594286


# RNN + Atención


## 3) Diseñe un clasificador con alguna RNN con atención que codifique cada perfil de usuario con todos sus tweets y clasifique.


De nuevo haciendo uso del código de Luis Pardo.

La sintaxis del modelo es similar al anterior pero se añade un módulo de atención. El modulo de atención toma los vectores de salida $h_t$ de la GRU y calcula una representación $s$ como suma ponderada:

$$ s = \sum_t \alpha_t h_t,$$

donde

\begin{align*}
    u_{t} &= \tanh(Wh_{t}+b),\\
    \alpha_{t} &= \frac{\exp(u_t^Tu)}{\sum_i\exp(u_{i}^Tu)}.
\end{align*}


In [ ]:
class AttnModule(nn.Module):
    def __init__(self, input_size, attn_hidden_size=128):
        '''
        input:
            input_size: tamaño de la capa oculta de la GRU.
            attn_hidden_size: tamaño de la capa oculta.
        '''
        super(AttnModule, self).__init__()
        self.fc1 = nn.Linear(input_size, attn_hidden_size)
        self.fc2 = nn.Linear(attn_hidden_size, 1, bias=False)

    def forward(self, seq, lengths):
        '''
        input:
            seq: secuencia de vectores ocultos de la GRU.
            lengths: número de palabras en cada observación.
        '''
        x = pad_packed_sequence(seq)[0]
        seq_len, batch_size, nhid = x.size()
        u = self.fc1(x.view(batch_size*seq_len, nhid))
        u = torch.tanh(u)
        scores = self.fc2(u)
        scores = scores.view(seq_len, batch_size, 1)
        # Asigna -100 a las posiciones con padding para que no sean consideados en la atención.
        scores = nn.utils.rnn.pack_padded_sequence(scores, lengths=lengths,enforce_sorted=False)
        scores = nn.utils.rnn.pad_packed_sequence(scores, padding_value=-100)[0]
        scores = F.softmax(scores, dim=0)
        scores = scores.transpose(0,1)
        x = x.transpose(0,1).transpose(1,2)
        x = torch.bmm(x, scores)
        return x.squeeze(2), scores

In [ ]:
class AttnRNN(nn.Module):
    def __init__(self, input_size=100, hidden_size=128, num_layers=1,
                 bidirectional=False, emb_mat=None, dense_hidden_size=256,
                 attn_hidden_size=128):
        super(AttnRNN, self).__init__()
        self.embeddings = nn.Embedding.from_pretrained( torch.FloatTensor(emb_mat), freeze=False)
        self.gru = nn.GRU(input_size=input_size, hidden_size=hidden_size, 
                          num_layers=num_layers, bidirectional=bidirectional)
        directions = 2 if bidirectional else 1
        self.attn = AttnModule(input_size=hidden_size*directions)
        self.classifier = nn.Sequential(\
                            nn.Linear(hidden_size*directions, dense_hidden_size),
                            nn.BatchNorm1d(dense_hidden_size),
                            nn.ReLU(),
                            nn.Linear(dense_hidden_size, 2))
        
    def forward(self, input_seq, lengths):
        x = self.embeddings(input_seq)
        x = x.split(lengths.tolist())
        x = pad_sequence(x)
        x = pack_padded_sequence(x, lengths, enforce_sorted=False)
        output, hn = self.gru(x)
        x, scores = self.attn(output, lengths)
        x = self.classifier(x)
        return x, scores.detach()

In [ ]:
lr = 0.0001
epochs = 10
device = torch.device('cuda')

In [ ]:
model = AttnRNN(emb_mat=train_dataset.emb_mat, bidirectional=False).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
weight = train_dataset.get_weights().to(device)
criterion = nn.NLLLoss(weight = weight)

Entrenamiento del modelo:

In [ ]:
best_val_f1 = 0
for epoch in range(epochs):
    for data in tqdm(train_dataloader):
        torch.cuda.empty_cache()
        optimizer.zero_grad()
        seq, seq_len, labels, _ = data
        seq, labels = seq.to(device), labels.to(device)
        output, _ = model(seq, seq_len)
        output = F.log_softmax(output, dim=1)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
    
    #Evalúa los modelos en los conjuntos de entrenamiento y valuación
    train_loss, train_f1, _, _, _, train_accuracy = eval_model(model, train_dataloader, criterion, device)
    test_loss, test_f1, _, _, _, test_accuracy = eval_model(model, test_dataloader, criterion, device)
    print('epoch: %d'%(epoch))
    print('train_loss: %5f | train_f1: %5f | train_acc: %5f'%(train_loss, train_f1, train_accuracy)) 
    print('test_loss: %5f |  test_f1: %5f | test_acc: %5f'%(test_loss, test_f1, test_accuracy)) 
    if test_f1>best_val_f1:
        best_val_f1=test_f1
        best_state_dict=copy.deepcopy(model.state_dict())

  0%|          | 0/420 [00:00<?, ?it/s]

  0%|          | 0/420 [00:00<?, ?it/s]

  0%|          | 0/280 [00:00<?, ?it/s]

epoch: 0
train_loss: 0.535987 | train_f1: 0.747441 | train_acc: 0.747381
test_loss: 0.559744 |  test_f1: 0.748070 | test_acc: 0.743571


  0%|          | 0/420 [00:00<?, ?it/s]

...

epoch: 8
train_loss: 0.257682 | train_f1: 0.915365 | train_acc: 0.913571
test_loss: 0.490361 |  test_f1: 0.794968 | test_acc: 0.784643


  0%|          | 0/420 [00:00<?, ?it/s]

  0%|          | 0/420 [00:00<?, ?it/s]

  0%|          | 0/280 [00:00<?, ?it/s]

epoch: 9
train_loss: 0.223733 | train_f1: 0.934654 | train_acc: 0.934048
test_loss: 0.480698 |  test_f1: 0.792492 | test_acc: 0.786786


# Atención Jeráquica

## 4) Diseñe un clasificador con alguna RNN con atención en jerárquia (como en el paper adjunto) que codifique cada perfil de usuario con todos sus tweets y clasifique. El primer nivel intuitivamente recibe embeddings de palabras, el segundo de tweets.

El módulo de antención es el mismo que en el caso pasado, de nuevo tomada del código de Luis Pardo.

In [ ]:
class AttnModule(nn.Module):
    def __init__(self, input_size, attn_hidden_size=128, output_size = 1):
        '''
        input:
            input_size: tamaño de la capa oculta de la GRU.
            attn_hidden_size: tamaño de la capa oculta.
        '''
        super(AttnModule, self).__init__()
        self.fc1 = nn.Linear(input_size, attn_hidden_size)
        self.fc2 = nn.Linear(attn_hidden_size, output_size, bias=False)

    def forward(self, seq, lengths):
        '''
        input:
            seq: secuencia de vectores ocultos de la GRU.
            lengths: número de palabras en cada observación.
        '''
        x = pad_packed_sequence(seq)[0]
        seq_len, batch_size, nhid = x.size()
        u = self.fc1(x.view(batch_size*seq_len, nhid))
        u = torch.tanh(u)
        scores = self.fc2(u)
        scores = scores.view(seq_len, batch_size, 1)
        # Asigna -100 a las posiciones con padding para que no sean consideados en la atención.
        scores = nn.utils.rnn.pack_padded_sequence(scores, lengths=lengths,enforce_sorted=False)
        scores = nn.utils.rnn.pad_packed_sequence(scores, padding_value=-100)[0]
        scores = F.softmax(scores, dim=0)
        scores = scores.transpose(0,1)
        x = x.transpose(0,1).transpose(1,2)
        x = torch.bmm(x, scores)
        return x.squeeze(2), scores

Para el caso jerárquico lo que cambia es el orden y jerarquía en la que se agrega la atención:


La siguiente es una modificación a la clase AttRNN() del inciso anterior.

In [ ]:
class Hierarchical_Attn(nn.Module):
    def __init__(self, input_size=100, hidden_size=128, num_layers=1,
                 bidirectional=False, emb_mat=None, dense_hidden_size=256,
                 attn_hidden_size=128):
        super(Hierarchical_Attn, self).__init__()
        #Embeddings
        self.embeddings = nn.Embedding.from_pretrained(torch.FloatTensor(emb_mat), freeze=False)
        #-----------------Palabras---------------------- 
        self.words_gru = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, bidirectional=bidirectional)
        directions = 2 if bidirectional else 1
        #Atención en las palabras
        self.words_attn = AttnModule(input_size=hidden_size*directions)
        self.att_dim1 = 128
        #--------------Tweets o sentencias----------------------------
        self.sentence_gru = nn.GRU(input_size=self.att_dim1, hidden_size=hidden_size, 
                          num_layers=num_layers, bidirectional=bidirectional, batch_first=True)
        directions = 2 if bidirectional else 1
        #-----Atención de los tweets------------------
        self.sentence_attn = AttnModule(input_size=hidden_size*directions, output_size = 128)
        self.att_dim2 = 128
        
        #-------Classifier-------------
        self.fc1 = nn.Linear(hidden_size * num_layers, self.att_dim2)
        self.fc2 = nn.Linear(self.att_dim1, 1, bias=False) 
        self.fc = nn.Sequential(nn.Linear(hidden_size,2))

        
    def forward(self, input_seq, lengths):
        x = self.embeddings(input_seq)
        x = x.split(lengths.tolist())
        x = pad_sequence(x)
        x = pack_padded_sequence(x, lengths, enforce_sorted=False)
        output, hn = self.words_gru(x)
        x, scores1 = self.words_attn(output, lengths)
        x          = x.unsqueeze(0)
        
        x, hn = self.sentence_gru(x)
        seq_len, batch_size, nhid = x.size()
        u = self.fc1(x.view(batch_size*seq_len, nhid))
        u = torch.tanh(u)
        scores = self.fc2(u)
        scores = scores.view(seq_len, batch_size, 1)
        scores = F.softmax(scores, dim=0)
        scores = scores.transpose(0,1)
        x = x.transpose(0,1).transpose(1,2)
        x = torch.bmm(x, scores)
        x = x.squeeze(2)
        x = self.fc(x)        
        return x, scores1.cpu().detach().numpy(), scores.cpu().detach().numpy()

In [ ]:
lr = 0.0001
epochs = 10
weight_decay=0.0001
device = torch.device('cuda')
eval_epoch = 1

In [ ]:
torch.cuda.set_device(1)

In [ ]:
model = Hierarchical_Attn(emb_mat=train_dataset.emb_mat, bidirectional=False).to(device)
optimizer = optim.Adam(model.parameters(),)
weight = train_dataset.get_weights().to(device)
criterion = nn.NLLLoss(weight = weight)

Entrenamiento:

In [ ]:
def eval_model(model, dataloader, criterion, device):
    '''Función para evaluar el modelo.'''
    with torch.no_grad():
        model.eval()
        losses = []
        preds = torch.empty(0).long()
        targets = torch.empty(0).long()
        scores_list = []
        words_list = []
        pred_list = []
        for data in tqdm(dataloader):
            torch.cuda.empty_cache()
            seq, seq_len, labels, words = data
            seq, labels = seq.to(device), labels.to(device)
            output, scores, _ = model(seq, seq_len)
            output = F.log_softmax(output, dim=1)
            loss = criterion(output, labels)
            losses.append(loss.item())
            predictions = F.log_softmax(output, dim=1).argmax(1)
            preds = torch.cat([preds, predictions.cpu()], dim=0)
            targets = torch.cat([targets, labels.cpu()], dim=0)
            if scores is not None:
                pred_list += predictions.tolist()
                scores = scores.squeeze(2).tolist()
                scores_list += scores
                words_list += words

        model.train()
        preds = preds.numpy()
        targets = targets.numpy()
        f1 = f1_score(targets, preds, average='binary')
        acc = accuracy_score(targets, preds)
        return np.mean(losses), f1, scores_list, words_list, pred_list, acc

In [ ]:
best_val_f1 = 0
for epoch in range(epochs):
    for data in tqdm(train_dataloader):
        torch.cuda.empty_cache()
        optimizer.zero_grad()
        seq, seq_len, labels,_ = data
        seq, labels = seq.to(device), labels.to(device)
        output, _, _  = model(seq, seq_len)
        output = F.log_softmax(output, dim=1)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
    
    #Evalúa los modelos en los conjuntos de entrenamiento y valuación
    train_loss, train_f1, _, _, _, train_accuracy = eval_model(model, train_dataloader, criterion, device)
    test_loss, test_f1, _, _, _, test_accuracy = eval_model(model, test_dataloader, criterion, device)
    print('epoch: %d'%(epoch))
    print('train_loss: %5f | train_f1: %5f | train_acc: %5f'%(train_loss, train_f1, train_accuracy)) 
    print('test_loss: %5f |  test_f1: %5f | test_acc: %5f'%(test_loss, test_f1, test_accuracy)) 
    if test_f1>best_val_f1:
        best_val_f1=test_f1
        best_state_dict=copy.deepcopy(model.state_dict())

  0%|          | 0/420 [00:00<?, ?it/s]

  0%|          | 0/420 [00:00<?, ?it/s]

  0%|          | 0/280 [00:00<?, ?it/s]

epoch: 0
train_loss: 0.520134 | train_f1: 0.707048 | train_acc: 0.746667
test_loss: 0.609721 |  test_f1: 0.625521 | test_acc: 0.679286


  0%|          | 0/420 [00:00<?, ?it/s]

...

epoch: 8
train_loss: 0.006726 | train_f1: 0.997856 | train_acc: 0.997857
test_loss: 1.160689 |  test_f1: 0.750533 | test_acc: 0.749286


  0%|          | 0/420 [00:00<?, ?it/s]

  0%|          | 0/420 [00:00<?, ?it/s]

  0%|          | 0/280 [00:00<?, ?it/s]

epoch: 9
train_loss: 0.010416 | train_f1: 0.998334 | train_acc: 0.998333
test_loss: 1.280812 |  test_f1: 0.747023 | test_acc: 0.749643


# Visualización de la atención

##  5) Compare y discuta el rendimiento de los modelos anteriores. Para el punto 4, construya visualizaciones como en la Figura 6 del paper de atención jerárquica y discuta.



Respecto a los modelos, notamos que el caso de RNN + atención simple obtuvo el mejor accuracy, en comparación al modelo sin atención que pareciera no haber aprendido mucho, sin embargo, puede ser que sólo sea una cuestión de modificar los parámetros de la arquitectura. Finalmente para el caso jerárquico, notemos que se obtuvo un .73 de accuracy. 


Ahora usamos el código de Luis Pardo para visualizar la atención:


In [ ]:
from IPython.display import display, HTML
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
def colorize(words, color_array):
    '''
        Función para visuzalizar la atención, tomada de https://gist.github.com/ihsgnef/f13c35cd46624c8f458a4d23589ac768,
    '''
    # words is a list of words
    # color_array is an array of numbers between 0 and 1 of length equal to words
    cmap = matplotlib.cm.get_cmap('Reds')
    template = '<span class="barcode"; style="color: black; background-color: {}">{}</span>'
    colored_string = ''
    for word, color in zip(words, color_array):
        color = matplotlib.colors.rgb2hex(cmap(color)[:3])
        colored_string += template.format(color, '&nbsp' + word + '&nbsp')
    return colored_string

In [ ]:
att = np.linspace(0,1,50)
p = [' ']*50
s = colorize(p, att)
# to display in ipython notebook
display(HTML(s))

In [ ]:
model.load_state_dict(best_state_dict)
train_loss, train_f1, train_scores, train_words, train_pred ,train_accuracy = eval_model(model, train_dataloader, criterion, device)
test_loss, test_f1, test_scores, test_words, test_pred, test_accuracy = eval_model(model, test_dataloader, criterion, device)
print('train_loss: %5f | train_f1: %5f'%(train_loss, train_f1)) 
print('test_loss: %5f | test_f1: %5f'%(test_loss, test_f1)) 

  0%|          | 0/420 [00:00<?, ?it/s]

  0%|          | 0/280 [00:00<?, ?it/s]

train_loss: 0.191489 | train_f1: 0.943025
test_loss: 0.464645 | test_f1: 0.799453


In [ ]:
max_attn = [np.max(scores) for scores in train_scores]
maxi = np.flip(np.argsort(max_attn))

In [ ]:
for j in range(10):
    i = maxi[j]
    s = colorize(train_words[i], train_scores[i][:len(train_words[i])])
    # to display in ipython notebook
    category = 'Agresivo' if train_pred[maxi[j]]==1 else 'No agresivo'
    print('Categoría predicha: %s'%(category))
    display(HTML(s))

Categoría predicha: No agresivo


Categoría predicha: No agresivo


Categoría predicha: Agresivo


Categoría predicha: No agresivo


Categoría predicha: Agresivo


Categoría predicha: No agresivo


Categoría predicha: Agresivo


Categoría predicha: No agresivo


Categoría predicha: Agresivo


Categoría predicha: Agresivo


# Preguntas

## 6 Comparación con los resultados sin Deep Learning.

    La estrategia del artículo usa en gran medida ponderaciones con significado referente a el objetivo de perfilar autores, pues incluso la representación de los datos (USR) es basada en cómo los autores se asemejan entre ellos por la relación que tienen: si ambos escriben en español, con emojis, o carácteres especiales, por dar algunos ejemplos. Mientras que los modelos profundos de esta tarea son casi que una caja negra de clasificación, dónde sólo hubo un pequeño preprocesamiento de los datos, meramente para quitar ciertos tokens, por lo tanto las arquitecturas no se prestan mucho para la interpretación respecto al perfilado de autores en tanto detalle.


## 6.1 ¿Le ganó a la mejor propuesta de representaciones distribucionales según lo reportado por el profesor? Si no fue así: ¿Le ganó a la Bolsa de Términos con SVM?

    Incluso con el modelo jerárquico el accuracy apenas logró llegar al 80%, mientras que con lo reportado en el artículo del profesor con su estrategia lograron lo mismo con herramientas de aprendizaje máquina. Por otro lado, los modelos de atención quedan a la par de la BoT con SVM.


## 6.2 ¿Por que cree que NO o SI le ganó? Discuta.

    Yo creo que no le ganó, si tomamos en consideración lo que tardaron los modelos de atención y la necesidad de encontrar parámetros para la red que dieran el mejor resultado, no tiene comparación con un modelo SVM y las bolsa de términos (BoT). De hecho, en el preprocesamiento, podemos observar que lo que se realizó con los tweets es muy parecido a la bolsa de términos, en donde partimos cada tweet en tokens (o términos), a diferencia que usamos su representación vectorial en vez de una matricial con ciertas ponderaciones para cada término. 

6.3 ¿Qué usó el primer lugar de la competencia? (Basile, et al.) ( Consulte: http://ceur-ws.org/Vol-1866/ )

    Hicieron uso de n-gramas de caracteres y palabras como entrada, y como clasificador usaron una SVM.

6.4 ¿Cuántos y que competidores usaron deep learning? En una o dos oraciones escriba qué hicieron

    Un total de 5 equipos hicieron uso de Deep Learning. Para el caso de clasificar el género de autor en Hombre/Mujer los otros participantes hicieron uso de CNN con matrices de N-gramas (Nils Schaetti) logrando un accuracy de .7107, mientras que otros (Miura et al) mediante el uso de  RNN con atención y FC NN lograron 81.18, comparado al 72.57 con una sola red RNN y atención (Don Kodiyan et al).  También hicieron uso de una arquitectura  LSTM pero no reportaron su accuracy obtenido (Bakhteev et al). Finalmente se obtuvo un 75.5 mediante el uso de DAN (Deep Averaging Network) (Salvador et al). Los artículos citados se encuentran a continuación preservando el orden en el que fueron referenciados.

    1. http://ceur-ws.org/Vol-1866/paper_80.pdf
    2. http://ceur-ws.org/Vol-1866/paper_90.pdf
    3. http://ceur-ws.org/Vol-1866/paper_77.pdf
    4. http://ceur-ws.org/Vol-1866/paper_68.pdf
    5. http://ceur-ws.org/Vol-1866/paper_192.pdf
